In [1]:
import pandas as pd
import numpy as np
pd.set_option("string_storage", "pyarrow")

## Load patstat to use patent family id in patentsview

In [ ]:
patstat = pd.read_parquet('../patstat/tls201.parquet', columns=['appln_id','docdb_family_id'])
patstat.head()

,appln_id,docdb_family_id
0,0,0
1,1,8554171
2,2,27517085
3,3,7915918
4,4,22889365


In [3]:
patstat= patstat.astype('Int64')
patstat.dtypes

appln_id           Int64
docdb_family_id    Int64
dtype: object

In [ ]:
## Use concordance at https://rawpatentdata.blogspot.com/2019/07/patstat-patentsview-concordance-update.html for the match
pv2patstat = pd.read_csv(
    "patentid_applnid.txt",
    sep="\t",
    engine="pyarrow",
    dtype={"patent_id": "string[pyarrow]", "APPLN_ID": 'Int64'},
)
pv2patstat.columns=['patent_id','appln_id']
pv2patstat.head()

,patent_id,appln_id
0,D584876,55754619
1,D584877,55754620
2,D584878,55754621
3,D584879,55754622
4,D584880,55754623


In [ ]:
patvecdf = pd.read_parquet('../patent_text/g_patent_title_abstract.parquet',columns=['patent_id'])
patvecdf.tail()

,patent_id
8436242,T998013
8436243,T998014
8436244,T999001
8436245,T999002
8436246,T999003


In [6]:
patvecdf.shape

(8436247, 1)

In [8]:
patvecdf = patvecdf.merge(pv2patstat, how='left')
patvecdf.tail()

,patent_id,appln_id
8436242,T998013,47886869
8436243,T998014,45929246
8436244,T999001,46589044
8436245,T999002,46955122
8436246,T999003,47470096


In [9]:
patvecdf = patvecdf.merge(patstat, how='left')
patvecdf.head()

,patent_id,appln_id,docdb_family_id
0,10000000,469404924,55456961
1,10000001,457553560,56163195
2,10000002,471123001,53493659
3,10000003,445333879,48045399
4,10000004,458352386,56234461


In [10]:
patvecdf.shape

(8436247, 3)

In [11]:
patvecdf.docdb_family_id.isna().sum()

1814480

In [12]:
patvecdf.nunique()

patent_id          8436247
appln_id           6621618
docdb_family_id    5734067
dtype: int64

In [13]:
patvecdf.to_parquet('data/patvecdf.parquet',index=False)